A simple file to do some debugging on the Python/Rust interface.

In [1]:
import ilqr

In [2]:
s = ilqr.ILQRSolver(1, 1, [[]], [[]])
print(s)

ILQRSolver { state_dim: 1, control_dim: 1, Q: VecStorage { data: [], nrows: Dyn(1), ncols: Dyn(0) }, R: VecStorage { data: [], nrows: Dyn(1), ncols: Dyn(0) } }
